In [1]:
#Procedimiento realizado a la tabla sucursal 
#   1.Se borran los datos repetidos si existen y los valores null
#   2.Se modifica el id de la tabla para lograr que sea un int:
#       2.1. Se verifica los valores maximos esperados
#       2.2. Se crea el nuevo Id a partir de los valores del Id existente
#       (Nota el nuevo id consta de los siguientes digitos XXYYZZZZ:
#           XX-comercioId, YY-banderaId, ZZZZ-sucursalId  )


#Se carga las librerias a usar
import pandas as pd
import xlrd

#Se carga la tabla desde el archivo .csv
df_sucursal = pd.read_csv('Datasets\sucursal.csv',sep = ',', decimal = '.',encoding='latin1')

# Se borran valores repetidos
df_sucursal.drop_duplicates(inplace=True)

#Se borran valores nulos
df_sucursal.dropna(inplace=True)

#Se convierte el codigo string en una lista con 
# los subcodigos (comercioId-banderaId-sucursalId.) 
Ids =[]
for id_Tex in df_sucursal["id"]:
    Ids.append(id_Tex.split("-"))   

# Se verifica el maximo valor para cada uno de los 
#   sub codigos y si existen incongruencia como letras
for i in range(3):
    error=0
    max = 0
    for Idcompuesta in Ids:
        try:
            if int(Idcompuesta[i]) > max:
                max = int(Idcompuesta[i])
        except ValueError:
            error = 1
    #print("Verificacion de posibles errores en subcodigos:",i, max, error)


#Se procede a crear la nueva columna con los nuevos Id_Sucursal
Id_Sucursal =[]
for Idcompuesta in Ids:
    num = int(Idcompuesta[0])*10000*100  +  int(Idcompuesta[1])*10000 + int(Idcompuesta[2])
    Id_Sucursal.append(num)

#Se inserta la nueva columna y se borra la anterior
df_sucursal.insert(0,"Id_Sucursal", Id_Sucursal, allow_duplicates=False)
df_sucursal.drop("id",axis=1,inplace=True)


#Se exporta la base de datos a MySQL
import sqlalchemy as db
database_username='root'            # Nombre de cliente en MySQL Workbrench
database_password='JCSR.MySQL123'   # Contraseña de MySQL Workbrench
database_ip='localhost'
database_name='ETL_Lab01'           # Nombre de Base de datos a donde nos conectaremos (se debio crear primero en Mysql)
database_conection = db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}',
                                    encoding='latin1')
coneccion=database_conection.connect()
df_sucursal.to_sql('sucursal',coneccion, index=False)
print("Carga de tabla 'sucursal' a MySQL completa")


import pymysql
conexion = pymysql.connect(
   host = 'localhost',
   user = 'root',
   passwd='JCSR.MySQL123',
   db='ETL_Lab01'           #LA base de datos se debio crear primero en Mysql
)
cursor = conexion.cursor()
try:
    #Se borra la base de datos si existe para cargarla nuevamente
    cursor.execute("alter table sucursal add primary key (Id_sucursal)")
finally:
    conexion.close()


Carga de tabla 'sucursal' a MySQL completa
